# Pylops-distributed - Solvers

In this notebook we investigate the use of scipy solvers with distributed operators. 
We will also create a simple CG solver to compare the results and performance

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import skfmm
import dask.array as da
import pylops
import pylops_distributed

from scipy.sparse.linalg.interface import MatrixLinearOperator, aslinearoperator 
from scipy.sparse import csr_matrix, vstack
from scipy.linalg import lstsq, solve
from scipy.sparse.linalg import LinearOperator, cg, lsqr, gmres
#
from scipy.signal import convolve, filtfilt

In [2]:
client = pylops_distributed.utils.backend.dask()
client

Client Scheduler: inproc://10.10.154.93/3745/1 Dashboard: http://localhost:54702/status,Cluster Workers: 1 Cores: 1 Memory: 8.59 GB


## CG solver

In [3]:
def conjgrad(A, b, x0, niter):
    'Conjugate-gradient algorithm for minimizing |A x - dat|^2'
    x = x0
    r = b-A._matvec(x).ravel()
    d = r.copy()
    k = r.dot(r)
    k0 = k
    for iter in range(niter):
        a = r.dot(r) / d.dot(A._matvec(d).ravel())
        x = x + a*d
        rnew = r - a*A._matvec(d).ravel()
        k = rnew.dot(rnew)
        b = rnew.dot(rnew) / r.dot(r)
        d = rnew + b*d;
        r = rnew.copy()
    return x

Let's just try out the solver with a matrix 

In [4]:
n = 10
x = np.ones(n)

A = np.random.randn(n, n)
A = np.dot(A.T, A)
print('eigs', np.linalg.eig(A)[0])

#A = MatrixLinearOperator(A)
Aop = aslinearoperator(A)

y = Aop.matvec(x)
xinv_sp = cg(Aop, y, maxiter=n)[0]
xinv = conjgrad(Aop, y, np.zeros_like(x), n)
print(xinv_sp)
print(xinv)

eigs [2.66828020e+01 2.11109110e+01 1.63380312e+01 1.23872079e+01
 9.13557102e+00 6.63534001e+00 3.13656650e+00 1.86068746e+00
 1.08849168e-03 2.69793992e-01]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


And with the LinearOperator

In [5]:
Ada = da.from_array(A, chunks=(n//2, n//2))
Aop = pylops_distributed.MatrixMult(Ada, compute=(True, True))
yy = Aop * da.ones(n, chunks=(n//2,))

xinv_sp = cg(Aop, yy, maxiter=n)[0]
xinv = conjgrad(Aop, yy, np.zeros_like(x), n)
print(xinv_sp)
print(xinv)
print(xinv.compute())

[1.         0.99999999 1.00000001 1.         1.         1.
 1.         1.00000001 1.         1.        ]
dask.array<add, shape=(10,), dtype=float64, chunksize=(5,)>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
%timeit -n 3 -r 1 cg(Aop, y, maxiter=n)[0]
%timeit -n 3 -r 1 conjgrad(Aop, y, np.zeros_like(x), n).compute()

658 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
1.03 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


**Note:** in order to allow lazy evaluation of mat-vec products in a solver (leading to a dask array ``xinv``), it is required to use ``_matvec`` and  ``_rmatvec`` instead of 
``matvec`` and  ``rmatvec``. However scipy solvers use the latter... 

Moreover the graph creation when allowing lazy evaluation seem to be the bottleneck...

what is the best way forward?